In [31]:
!uv pip install langchain
!uv pip install langchain-openai
!uv pip install mem0ai
!uv pip install python-dotenv
!uv pip install redis redisvl

Now using Node v24.8.0 (npm 11.6.0) ~/.local/share/nvm/v24.8.0/bin/node
Audited 1 package in 5ms
Now using Node v24.8.0 (npm 11.6.0) ~/.local/share/nvm/v24.8.0/bin/node
Audited 1 package in 3ms
Now using Node v24.8.0 (npm 11.6.0) ~/.local/share/nvm/v24.8.0/bin/node
Audited 1 package in 4ms
Now using Node v24.8.0 (npm 11.6.0) ~/.local/share/nvm/v24.8.0/bin/node
Audited 1 package in 2ms
Now using Node v24.8.0 (npm 11.6.0) ~/.local/share/nvm/v24.8.0/bin/node
Resolved 14 packages in 2.71s                                        
⠹ Preparing packages... (0/5)                                                   
⠹ Preparing packages... (0/5)--------------     0 B/48.41 KiB           
⠹ Preparing packages... (0/5)--------------     0 B/48.41 KiB           
python-ulid          ------------------------------     0 B/11.31 KiB
⠹ Preparing packages... (0/5)--------------     0 B/48.41 KiB           
python-ulid          ------------------------------ 11.31 KiB/11.31 KiB
⠹ Preparing packages... (0/5

In [1]:
import os
import dotenv
import logging
import uuid
from datetime import datetime
from typing import Dict, List
from openai import OpenAI
from mem0 import Memory
from mem0.configs.base import MemoryConfig
from mem0.embeddings.configs import EmbedderConfig
from mem0.vector_stores.configs import VectorStoreConfig
from mem0.llms.configs import LlmConfig
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



In [2]:
logging.basicConfig(level=logging.INFO,format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
from urllib3 import response


class ExpressCustomerService:
    def __init__(self):
        self.api_key = self._get_api_key()
        self.base_url = self._get_base_url()

        self.openai_client = None
        self.llm = None
        self.memory = None
        self.prompt = None

        self._initialize_components()
    #
    def _get_api_key(self) -> str:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError(
                "未找到DASHSCOPE_API_KEY环境变量.\n"
                "请设置环境变量：export DASHSCOPE_API_KEY=<你的API密钥>"
            )
        logger.info("API密钥已加载")
        return api_key

    def _get_base_url(self) -> str:
        base_url = os.getenv("OPENAI_BASE_URL")
        if not base_url:
            raise ValueError(
                "未找到OPENAI_BASE_URL环境变量.\n"
                "请设置环境变量：export OPENAI_BASE_URL=<你的API基础URL>"
            )
        logger.info("API基础URL已加载")
        return base_url

    def _is_valid_openai_client(self) -> bool:
        try:
            response = self.openai_client.chat.completions.create(
                model="qwen3-max",
                messages=[{"role": "user", "content": "测试连接"}],
                max_tokens=10,
            )
            logger.info("API连接成功")
            return True
        except Exception as e:
            logger.error(f"API连接失败: {str(e)}")
            return False

    def _initialize_components(self):
        try:

            self.openai_client = OpenAI(api_key=self.api_key, base_url=self.base_url)
            logger.info("OpenAI客户端初始化成功")

            if not self._is_valid_openai_client():
                raise ValueError("API初始化失败")

            self.llm = ChatOpenAI(
                model="qwen3-max",
                temperature=0.3,
                api_key=self.api_key,
                base_url=self.base_url,
            )
            logger.info("LLM初始化成功")

            self.memory = Memory(
                config=MemoryConfig(
                    llm=LlmConfig(
                        provider="openai",
                        config={
                            "model": "qwen3-max",
                            "api_key": self.api_key,
                            "openai_base_url": self.base_url,
                        },
                    ),
                    embedder=EmbedderConfig(
                        provider="openai",
                        config={
                            "model": "text-embedding-v4",
                            "api_key": self.api_key,
                            "openai_base_url": self.base_url,
                        },
                    ),
                    # vector_store=VectorStoreConfig(
                    #     provider="redis",
                    #     config={
                    #         "collection_name": "mem0",
                    #         "embedding_model_dims": 1536,
                    #         "redis_url": "redis://localhost:6379",
                    #     },
                    # ),
                    vector_store=VectorStoreConfig(
                        provider="qdrant",
                        config={
                            "collection_name": "mem0",
                            "embedding_model_dims": 1536,
                            "url": "http://localhost:6333",  # 使用完整URL而不是host/port
                        },
                    ),
                )
            )
            logger.info("Memory初始化成功")

            self._initialize_prompt()

        except Exception as e:
            logger.error(f"初始化失败: {str(e)}")
            raise

    def _initialize_prompt(self):
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(
                    content=""""您是一位专业的快递行业智能客服助手。请使用提供的上下文信息来个性化您的回复，记住用户的偏好和历史交互记录。

您的主要职责包括：
1. 快递查询服务：帮助用户查询包裹状态、物流轨迹、预计送达时间
2. 寄件服务：提供寄件指导、价格咨询、时效说明、包装建议
3. 问题解决：处理快递延误、丢失、损坏等问题，提供解决方案
4. 服务咨询：介绍各类快递服务、收费标准、服务范围
5. 投诉建议：接收用户反馈，记录投诉信息并提供处理方案

回复时请保持：
- 专业、礼貌、耐心的服务态度
- 准确、及时的信息提供
- 个性化的服务体验
- 如果没有具体信息，可以基于快递行业常识提供建议

请用中文回复，语气亲切专业。"""
                ),
                MessagesPlaceholder(variable_name="context"),
                HumanMessage(content="{input}"),
            ]
        )

    def retrieve_context(self, user_query: str, user_id: str) -> List[Dict]:
        try:
            memories = self.memory.search(query=user_query, user_id=user_id)
            if memories and "results" in memories and memories["results"]:
                serialized_memories = " ".join(
                    [mem.get("memory", "") for mem in memories["results"]]
                )
            else:
                serialized_memories = "暂无相关历史记录"
            context = [
                {"role": "system", "content": f"相关历史信息：{serialized_memories}"},
                {"role": "user", "content": user_query},
            ]

            return context
        except Exception as e:
            logger.error(f"上下文检索失败: {str(e)}")

            return [
                {"role": "system", "content": "相关历史信息：暂无相关历史记录"},
                {"role": "user", "content": user_query},
            ]

    def generate_response(self, user_query: str, context: List[Dict]) -> str:
        try:
            chain = self.prompt | self.llm
            response = chain.invoke({"input": user_query, "context": context})
            return response.content
        except Exception as e:
            logger.error(f"生成响应失败: {str(e)}")
            return "很抱歉，我暂时无法回答这个问题。"

    def save_memory(self, user_query: str, assistant_response: str, user_id: str):
        try:
            interation = [
                {"role": "user", "content": user_query},
                {"role": "assistant", "content": assistant_response},
            ]
            self.memory.add(
                interation,
                user_id=user_id,
            )
            logger.info(f"用户{user_id}的交互记录已保存")
        except Exception as e:
            logger.error(f"保存交互记录失败: {str(e)}")

    def chat_with_user(self, user_id: str, user_query: str):
        try:
            context = self.retrieve_context(user_query, user_id)
            response = self.generate_response(user_query, context)
            self.save_memory(user_query, response, user_id)
            return response
        except Exception as e:
            logger.error(f"与用户{user_id}的交互失败: {str(e)}")
            return "很抱歉，我暂时无法回答这个问题。"

    def run_interactive_chat(self):
        print("=" * 60)
        print("欢迎使用能客服系统")
        print("=" * 60)
        print("我可以帮您处理各种快递相关问题：")
        print("输入 'exit' 或 'quit' 退出程序")
        print("=" * 60)

        user_id = str(uuid.uuid4())
        session_id = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}_{user_id}"
        print(f"当前客户ID: {user_id}")
        print(f"当前会话ID: {session_id}")
        print("-" * 60)
        while True:
            try:
                user_query = input("请输入您的查询内容：").strip()

                if user_query.lower() in ["exit", "quit"]:
                    print("感谢您的使用，再见！")
                    break
                if not user_query:
                    print("请输入有效的查询内容。")
                    continue
                print(f"用户问题: {user_query}\n")
                response = self.chat_with_user(session_id, user_query)
                print(f"智能客服：{response}\n")
            except KeyboardInterrupt:
                print("\n感谢您的使用，再见！")
                break
            except Exception as e:
                print(f"发生错误: {str(e)}")
                print("请稍后再试。")

In [4]:
def main():
    try:
        dotenv.load_dotenv()
        customer_service=ExpressCustomerService()
        customer_service.run_interactive_chat()
    except Exception as e:
        print(f"发生致命错误: {str(e)}")
        print("程序已终止。")


if __name__=="__main__":
    main()

2025-09-29 00:38:23,929 - INFO - API密钥已加载
2025-09-29 00:38:23,929 - INFO - API基础URL已加载
2025-09-29 00:38:24,029 - INFO - OpenAI客户端初始化成功
2025-09-29 00:38:25,960 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:38:25,964 - INFO - API连接成功
2025-09-29 00:38:26,036 - INFO - LLM初始化成功
/Users/kevin/dev/ai/homework/.venv/lib/python3.11/site-packages/mem0/utils/factory.py:189: DeprecationWarning: Function set_client is deprecated and will be removed in the next major release. Pass connection parameters in __init__.
  return vector_store_instance(**config)
2025-09-29 00:38:26,134 - INFO - Memory初始化成功


欢迎使用能客服系统
我可以帮您处理各种快递相关问题：
输入 'exit' 或 'quit' 退出程序
当前客户ID: 8c392238-778c-4fa2-a117-a9d1857f78bd
当前会话ID: session_20250929003826_8c392238-778c-4fa2-a117-a9d1857f78bd
------------------------------------------------------------
用户问题: 你好



2025-09-29 00:38:33,371 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings "HTTP/1.1 200 OK"
2025-09-29 00:38:36,160 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:38:38,149 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:38:38,151 - INFO - Total existing memories: 0
2025-09-29 00:38:38,153 - INFO - 用户session_20250929003826_8c392238-778c-4fa2-a117-a9d1857f78bd的交互记录已保存


智能客服：您好！欢迎使用快递服务咨询，我是您的智能客服助手。请问有什么可以帮您的吗？比如查询快递、寄件咨询、问题反馈等，都可以告诉我哦！😊

用户问题: 请查询 0001234567 状态



2025-09-29 00:38:59,211 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings "HTTP/1.1 200 OK"
2025-09-29 00:39:05,630 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:39:07,694 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:39:07,696 - INFO - Total existing memories: 0
2025-09-29 00:39:07,698 - INFO - 用户session_20250929003826_8c392238-778c-4fa2-a117-a9d1857f78bd的交互记录已保存


...

智能客服：您好！您提到“订单号错了吗？”，为了帮您准确核实，请您提供以下信息：

1. **完整的订单号**（或快递单号）  
2. **快递公司名称**（如顺丰、中通、京东物流等，如果知道的话）  
3. **您遇到的具体问题**（例如：查不到物流信息、物流信息异常、收件信息不符等）

有了这些信息，我可以立即帮您查询并确认订单号是否正确，或者是否存在其他问题。期待您的补充！

用户问题: 我之前说的订单号是多少？



2025-09-29 00:40:33,796 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings "HTTP/1.1 200 OK"
2025-09-29 00:40:40,024 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:40:43,104 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:40:43,105 - INFO - Total existing memories: 0
2025-09-29 00:40:43,107 - INFO - 用户session_20250929003826_8c392238-778c-4fa2-a117-a9d1857f78bd的交互记录已保存


智能客服：您好！在我们当前的对话中，您还没有提供过订单号。如果您之前在其他对话中提到过，我可能无法获取到那些信息（出于隐私和数据安全考虑，系统通常不会保留跨会话的历史记录）。

请您再次提供一下订单号，我会立即帮您查询物流状态或处理相关问题！📦

用户问题: 我不是有提供过 0001234567 吗？



2025-09-29 00:41:30,310 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings "HTTP/1.1 200 OK"
2025-09-29 00:41:34,729 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:41:37,546 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:41:39,293 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings "HTTP/1.1 200 OK"
2025-09-29 00:41:39,296 - INFO - Total existing memories: 0
2025-09-29 00:41:42,084 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-29 00:41:42,092 - INFO - {'id': '0', 'text': '用户提到曾提供过运单号 0001234567', 'event': 'ADD'}
2025-09-29 00:41:42,613 - ERROR - Error while loading data to Redis
Traceback (most recent call last):
  File "/Users/kevin/dev/ai/homework/.venv/lib/python3.11/site-packag

智能客服：您好！感谢您的提醒。不过在当前对话中，我暂时没有看到您之前提供过运单号 **0001234567** 的记录（可能是新会话或信息未同步）。为了更好地帮您查询快递状态，请您确认一下：

1. 这个单号是否正确？  
2. 您使用的是哪家快递公司？（例如：顺丰、中通、圆通、京东物流等）

一旦确认，我可以立即为您查询最新物流信息！📦

感谢您的使用，再见！
